In [1]:
!pip install transformers torch opencv-python nltk pillow

In [1]:
import nltk
nltk.data.clear_cache()



In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt', download_dir='/home/davidlimcher/projects/nowadays/synthetic_media/practice_6/myenv/nltk_data')


[nltk_data] Downloading package punkt to /home/davidlimcher/projects/n
[nltk_data]     owadays/synthetic_media/practice_6/myenv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.data.path = ['/home/davidlimcher/projects/nowadays/synthetic_media/practice_6/myenv/nltk_data']


In [4]:
import cv2
import torch
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize


# Инициализация модели и токенизатора
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Функция для получения описания кадра
def get_caption(image):
    inputs = feature_extractor(images=image, return_tensors="pt")
    pixel_values = inputs.pixel_values
    output_ids = model.generate(pixel_values)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

# Функция для извлечения существительных и глаголов из текста
def extract_nouns_verbs(captions):
    nouns, verbs = [], []
    for caption in captions:
        words = word_tokenize(caption)
        pos_tags = pos_tag(words)
        for word, tag in pos_tags:
            if tag.startswith('NN'):  # Noun
                nouns.append(word.lower())
            elif tag.startswith('VB'):  # Verb
                verbs.append(word.lower())
    return nouns, verbs




Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.1"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

In [5]:
# Открытие видео и обработка кадров
video_path = "RPReplay_Final1723701363.mp4"
cap = cv2.VideoCapture(video_path)
frame_count = 0
captions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % 5 == 0:  # Обрабатываем каждый 5-й кадр
        caption = get_caption(frame)
        captions.append(caption)
    
    frame_count += 1

cap.release()



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/davidlimcher/projects/nowadays/synthetic_media/practice_6/myenv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


In [6]:
# Извлечение и подсчет наиболее частых существительных и глаголов
nouns, verbs = extract_nouns_verbs(captions)
noun_freq = Counter(nouns)
verb_freq = Counter(verbs)

# Результаты
print("Наиболее частые существительные:", noun_freq.most_common(10))
print("Наиболее частые глаголы:", verb_freq.most_common(10))



Наиболее частые существительные: [('man', 77), ('shirt', 47), ('tie', 28), ('computer', 21), ('screen', 12), ('front', 12), ('picture', 8), ('light', 6), ('room', 5), ('camera', 4)]
Наиболее частые глаголы: [('is', 39), ('looking', 21), ('standing', 12), ('holding', 5), ('wearing', 2), ('using', 2), ('lighting', 1), ('playing', 1), ('cutting', 1)]


In [ ]:
# Сохранение анализа в README.md
with open("README.md", "w") as f:
    f.write("# Анализ видео\n\n")
    f.write("Наиболее частые существительные:\n")
    f.write(str(noun_freq.most_common(10)) + "\n\n")
    f.write("Наиболее частые глаголы:\n")
    f.write(str(verb_freq.most_common(10)) + "\n\n")
    f.write("### Гипотеза\n")
    f.write("Модель может не описывать видео полностью из-за ограничений в обучении на статичных изображениях, "
            "что ограничивает её понимание контекста и последовательности действий.\n")

In [4]:
!pip install torch torchvision opencv-python iopath pytorchvideo


DEPRECATION: Loading egg at /home/davidlimcher/projects/nowadays/synthetic_media/practice_6/myenv/lib/python3.12/site-packages/tensorboard-2.18.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/davidlimcher/projects/nowadays/synthetic_media/practice_6/myenv/lib/python3.12/site-packages/fairscale-0.4.13-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached av-13.1.0-cp312-cp312

In [6]:
import json
import cv2
import torch
from torchvision import transforms
from pytorchvideo.models.hub import slowfast_r50
from ultralytics import YOLO

# Загрузка классов действий
def load_class_names(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    class_names = {str(value): key.strip('"') for key, value in data.items()}
    return class_names

class_names = load_class_names('kinetics_classnames.json')
print(f"Загружено {len(class_names)} меток классов")

# Настройка моделей
action_model = slowfast_r50(pretrained=True).eval()
detection_model = YOLO("yolov8s.pt")

# Настройки видео
video_path = "RPReplay_Final1723701363.mp4"
output_path = "output_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Ошибка: Невозможно открыть видеофайл")
else:
    print("Видео успешно открыто")

width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
print(f"Видео параметры - Ширина: {width}, Высота: {height}, FPS: {fps}")

# Преобразование для модели
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor()
])

# Создание путей для SlowFast модели
def create_slowfast_input(input_tensor, alpha=4):
    slow_pathway = input_tensor[:, :, ::alpha, :, :]  # Уменьшение частоты кадров
    fast_pathway = input_tensor
    return [slow_pathway, fast_pathway]

# Обработка кадра для классификации действий
def process_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = transform(frame_rgb).unsqueeze(0)
    input_tensor = input_tensor.unsqueeze(2).repeat(1, 1, 32, 1, 1)
    slowfast_input = create_slowfast_input(input_tensor)

    with torch.no_grad():
        predictions = action_model(slowfast_input)

    probabilities = torch.nn.functional.softmax(predictions, dim=1)[0]
    top5_idx = probabilities.argsort(descending=True)[:5]
    actions = [(class_names.get(str(idx.item()), "Unknown"), probabilities[idx].item()) for idx in top5_idx]
    
    return actions

# Основной цикл для обработки видео
frame_count = 0
actions = []  # Хранение действий для отображения
detected_boxes = []  # Хранение рамок объектов для отображения
text_update_interval = int(fps / 2)  # Обновлять текст каждые полсекунды

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Конец видео или ошибка чтения кадра")
        break

    # Обновляем текст каждые полсекунды
    if frame_count % text_update_interval == 0:
        actions = process_frame(frame)  # Классификация действий на кадре
    
    # Обновляем рамки на каждом кадре
    results = detection_model(frame)
    detected_boxes = results[0].boxes.xyxy  # Получаем координаты рамок для обнаруженных объектов

    # Отрисовка рамок вокруг обнаруженных объектов и добавление меток действий
    for i, box in enumerate(detected_boxes):
        x1, y1, x2, y2 = map(int, box[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Рисуем рамку
        if i < len(actions):
            action, prob = actions[i]
            label = f"{action}: {prob:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)
    frame_count += 1

# Завершение записи
cap.release()
out.release()
cv2.destroyAllWindows()


Загружено 400 меток классов
Видео успешно открыто
Видео параметры - Ширина: 640, Высота: 480, FPS: 36.80933160964345

0: 480x640 1 person, 1 cup, 1 potted plant, 1 dining table, 1 laptop, 3.6ms
Speed: 1.1ms preprocess, 3.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cup, 1 potted plant, 1 dining table, 1 laptop, 3.4ms
Speed: 0.6ms preprocess, 3.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cup, 1 potted plant, 1 dining table, 1 laptop, 3.7ms
Speed: 0.5ms preprocess, 3.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cup, 1 potted plant, 1 dining table, 1 laptop, 3.8ms
Speed: 0.5ms preprocess, 3.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cup, 1 potted plant, 1 dining table, 1 laptop, 3.5ms
Speed: 0.7ms preprocess, 3.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 pers